```{contents}
```
## SequentialChain (LangChain)

### What SequentialChain Is

A **SequentialChain** is a **legacy LangChain construct** that allows you to **execute multiple chains one after another**, where the **output of one chain becomes the input of the next**.

> SequentialChain models a **fixed, linear workflow** with multiple LLM calls.

It was designed before **LCEL (Runnable-based pipelines)** existed.

---

### Why SequentialChain Exists

SequentialChain was created to:

* Break complex tasks into steps
* Reuse intermediate outputs
* Enforce execution order
* Reduce prompt complexity per step

Conceptually, it represents:

```
Chain A → Chain B → Chain C
```

---

### Core Concept

Each chain:

* Consumes input variables
* Produces output variables
* Passes outputs forward

LangChain manages the **shared state dictionary**.

---

### Simple SequentialChain Example

#### Step 1: Create Individual LLMChains



In [2]:
from langchain_classic.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

title_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Generate a short title for {topic}"
)

summary_prompt = PromptTemplate(
    input_variables=["title"],
    template="Write a 1-line summary for the title: {title}"
)

title_chain = LLMChain(llm=llm, prompt=title_prompt, output_key="title")
summary_chain = LLMChain(llm=llm, prompt=summary_prompt, output_key="summary")


C:\Users\sangouda\AppData\Local\Temp\ipykernel_42048\829087398.py:17: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  title_chain = LLMChain(llm=llm, prompt=title_prompt, output_key="title")




---

### Step 2: Combine Using SequentialChain



In [5]:
from langchain_classic.chains.sequential import SequentialChain

overall_chain = SequentialChain(
    chains=[title_chain, summary_chain],
    input_variables=["topic"],
    output_variables=["title", "summary"],
    verbose=True
)




---

#### Step 3: Invoke the Chain



In [6]:
result = overall_chain.invoke({"topic": "LangChain"})
print(result)




> Entering new SequentialChain chain...

> Finished chain.
{'topic': 'LangChain', 'title': '"LangChain: Empowering Language Models for Seamless Integration"', 'summary': '"LangChain enables effortless integration of language models into applications, enhancing their functionality and user experience."'}




Output:

```python
{
  "topic": "LangChain",
  "title": "Building with Language Models",
  "summary": "A concise overview of using language models in applications."
}
```

---

### How Data Flows Internally

```
Input: {"topic": "LangChain"}
   ↓
title_chain → {"title": "..."}
   ↓
summary_chain (uses title)
   ↓
Final Output
```

State is accumulated automatically.

---

### SimpleSequentialChain (Shortcut)

#### When Only One Output Is Needed



In [8]:
from langchain_classic.chains.sequential import SimpleSequentialChain

chain = SimpleSequentialChain(
    chains=[title_chain, summary_chain],
    verbose=True
)




Limitations:

* Single input
* Single output
* No variable control

---

### Common Use Cases

* Stepwise text generation
* Content pipelines
* Summarization → rewriting
* Classification → explanation
* Translation → formatting

---

### Limitations of SequentialChain

#### Structural Limitations

* ❌ Linear only (no branching)
* ❌ No parallelism
* ❌ Weak streaming support
* ❌ Poor async ergonomics
* ❌ Legacy abstraction

---

### SequentialChain vs LCEL (Recommended)

#### SequentialChain

```python
SequentialChain(chains=[a, b, c])
```

### LCEL Equivalent

```python
chain = a | b | c
```

LCEL:

* Is Runnable-based
* Supports streaming & async
* Allows branching and parallelism
* Is future-proof

---

### SequentialChain vs Agents

| Aspect      | SequentialChain | Agent   |
| ----------- | --------------- | ------- |
| Flow        | Fixed           | Dynamic |
| Tool usage  | ❌               | ✅       |
| Reasoning   | ❌               | ✅       |
| Determinism | High            | Lower   |

---

### When to Use SequentialChain (Today)

* Maintaining legacy code
* Learning LangChain basics
* Migrating step-by-step logic

---

### When NOT to Use SequentialChain

* New projects
* RAG pipelines
* Agents
* Production APIs
* Streaming applications

---

### Migration Example (SequentialChain → LCEL)

#### Before

```python
SequentialChain(chains=[chain1, chain2])
```

#### After

```python
chain = chain1 | chain2
```

Same logic, better execution model.

---

### Best Practices (If You Must Use It)

* Keep chains small
* Limit number of steps
* Avoid complex variable graphs
* Plan migration to LCEL

---

### Interview-Ready Summary

> “SequentialChain is a legacy LangChain abstraction for executing multiple chains in a fixed order, passing outputs between them. It has largely been replaced by LCEL, which offers better composability, streaming, async support, and production readiness.”

---

### Rule of Thumb

* **Linear legacy workflows → SequentialChain**
* **Modern pipelines → LCEL**
* **Dynamic reasoning → Agents or LangGraph**